PipeLine
- need to make a pdf, doc reader, docx, txt, reader
- Assume the files themselves are in a dynamno


Notes
- We have files from a dynamo db database, the files with be pdf, docx, whatever
- When stream assistant is called, we will do a keyword search of the data and then grab that chunk and feed it
    into the LLM. 
- Ok this is weird, I need to find the most efficent way of grabbing this data. I can query sort the database.
- Where would lance DB fit into this. 


Potential Pipelines
1. Keyword search the document metadata, grab the document, compute vectors on demand, find 


Sections of Pipeline, along with potential implementations
    1. Document storage only in Dynamo DB
        - Only keep documents in Dyanmo DB. Relevant document info loaded via document name search or metadata.
    2. Document storage with LLM summary. 
        - We can store the documents with a LLM summary of the documents that are created when the documents are loaded. This summary can be vectorized and searchd to see if document contains necessary info. 
        - We have the potential to get creative here, because we could treat each document as its own entity or we can create a kind of file system and summarize every page of every long document.
    3. Compute Vector database, store with documents.
        - Whenever documents are uploaded, we could trigger a process to compute and upload these vectors. These are then what is sorted.
        
    

Notes
- Lance DB
- Vector Database

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Example embeddings (just as random vectors for illustration)
embedding1 = [0.1, 0.2, 0.3]  # Embedding for sentence 1
embedding2 = [0.12, 0.19, 0.31]  # Embedding for a similar sentence
embedding3 = [-0.1, 0.3, -0.2]  # Embedding for a different sentence

# Calculate cosine similarity between embeddings
similarity12 = cosine_similarity([embedding1], [embedding2])[0][0]
similarity13 = cosine_similarity([embedding1], [embedding3])[0][0]

print(f"Similarity between sentence 1 and 2: {similarity12}")
print(f"Similarity between sentence 1 and 3: {similarity13}")

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

# Generation based on retrieved context
inputs = tokenizer("Context: The syllabus includes...", return_tensors="pt")
summary_ids = model.generate(inputs["input_ids"], max_length=50)
print(tokenizer.decode(summary_ids[0], skip_special_tokens=True))

In [ ]:
import faiss
import numpy as np

# Create a FAISS index (e.g., for 768-dimensional embeddings)
dimension = 768
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
embeddings = np.random.random((1000, dimension)).astype('float32')
index.add(embeddings)

# Query the index
query_embedding = np.random.random((1, dimension)).astype('float32')
distances, indices = index.search(query_embedding, k=5)

In [ ]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DenseRetriever, TransformersReader
from haystack.pipelines import ExtractiveQAPipeline

# Create a FAISS document store
document_store = FAISSDocumentStore(embedding_dim=768)

# Initialize retriever and reader
retriever = DenseRetriever(document_store=document_store, embedding_model="sentence-transformers/all-MiniLM-L6-v2")
reader = TransformersReader(model_name_or_path="deepset/roberta-base-squad2")

# Build a pipeline
pipeline = ExtractiveQAPipeline(reader, retriever)

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for your documents
document_embeddings = model.encode(["Document 1 text", "Document 2 text"])

# Generate an embedding for the query
query_embedding = model.encode("Your query text")
